# Cleaning Datasets
Now that I have all 3 datasets (funds, macro and market data), it's time to clean it. The cleaning process will be composed by the following steps:
1) Convert all variables to their correct datatype: a lot of our data comes in string format, when it should be a date or int/float.
2) Analyze null values: how many? does it impact our dataset?
3) Duplicates: does it make sense to have duplicates? if not, can I delete them? what's the impact?
4) Feature engineering: what other features/variables can I extract from out current data?

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from scipy import stats

## Financial funds cleaning

In [2]:
# Import dataset
cvm_daily_return = pd.read_csv('cvm_daily_return.csv')

# Transform 'DT_COMPTC' in datetime variable
cvm_daily_return['DT_COMPTC'] = pd.to_datetime(cvm_daily_return['DT_COMPTC'] , format='ISO8601')

# Drop columns
cvm_daily_return.drop(columns=['Unnamed: 0','TP_FUNDO','CAPTC_DIA','RESG_DIA','VL_TOTAL'], inplace=True)
cvm_daily_return.info()

C:\Users\anaak\AppData\Local\Temp\ipykernel_22028\2334502993.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  cvm_daily_return = pd.read_csv('cvm_daily_return.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26112510 entries, 0 to 26112509
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   CNPJ_FUNDO     object        
 1   DT_COMPTC      datetime64[ns]
 2   VL_QUOTA       float64       
 3   VL_PATRIM_LIQ  float64       
 4   NR_COTST       int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 996.1+ MB


In [3]:
# To calculate the funds return, we need the first and last day of the month
cvm_daily_return['year_month'] = cvm_daily_return['DT_COMPTC'].dt.to_period('M')

# Create separate dataframes to first and last day
first_day  = cvm_daily_return.groupby(['year_month', 'CNPJ_FUNDO']).min().reset_index()
last_day = cvm_daily_return.groupby(['year_month', 'CNPJ_FUNDO']).max().reset_index()

# Concat first and last day dataframes
cvm_return = pd.concat([first_day, last_day]).sort_values(by='DT_COMPTC').reset_index(drop=True)

# Clean memory
del cvm_daily_return

# Check values
cvm_return['DT_COMPTC'].unique()

<DatetimeArray>
['2020-01-01 00:00:00', '2020-01-02 00:00:00', '2020-01-03 00:00:00',
 '2020-01-06 00:00:00', '2020-01-07 00:00:00', '2020-01-08 00:00:00',
 '2020-01-09 00:00:00', '2020-01-10 00:00:00', '2020-01-13 00:00:00',
 '2020-01-14 00:00:00',
 ...
 '2024-08-06 00:00:00', '2024-08-07 00:00:00', '2024-08-08 00:00:00',
 '2024-08-09 00:00:00', '2024-08-12 00:00:00', '2024-08-13 00:00:00',
 '2024-08-14 00:00:00', '2024-08-15 00:00:00', '2024-08-16 00:00:00',
 '2024-08-19 00:00:00']
Length: 1177, dtype: datetime64[ns]

**Python show not all funds have the first register at the first day of the month. Is this possible?** 
- Yes. A fund only register the value of its assets once it's approved by CVM. Which means our starting date isn't the same for all funds, will depend on when their documents were processed.

**Is this bad for this analysis?**
-  No. Since I'm calculating the return (assets final value - assets start value), the number between those day won't bias the data. I can expect a small variation in value with smaller the window between those.

In [4]:
# Calculate the return
## Return is defined as the difference in quota's value at the month end and the beginning of the month
cvm_return.sort_values(by=['CNPJ_FUNDO','year_month','DT_COMPTC'], inplace=True, ascending=True)
cvm_return['quota_return'] = cvm_return.groupby(['CNPJ_FUNDO','year_month'])[['VL_QUOTA']].diff()
# cvm_return['assets_return'] = cvm_return.groupby(['CNPJ_FUNDO','year_month'])[['VL_PATRIM_LIQ']].diff()

In [5]:
# Analyze NA
print(cvm_return.isnull().sum())

# Check empty rows with null quota return values
cvm_return[cvm_return['quota_return'].isnull()== True]

year_month             0
CNPJ_FUNDO             0
DT_COMPTC              0
VL_QUOTA               0
VL_PATRIM_LIQ          0
NR_COTST               0
quota_return     1274328
dtype: int64


,year_month,CNPJ_FUNDO,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST,quota_return
27,2020-01,00.017.024/0001-53,2020-01-02,27.224496,1120401.09,1,NaN
45294,2020-02,00.017.024/0001-53,2020-02-03,27.259298,1119555.51,1,NaN
79726,2020-03,00.017.024/0001-53,2020-03-02,27.307906,1119544.44,1,NaN
114213,2020-04,00.017.024/0001-53,2020-04-01,27.380981,1118176.96,1,NaN
149732,2020-05,00.017.024/0001-53,2020-05-04,27.425677,1117875.24,1,NaN
...,...,...,...,...,...,...,...
2295906,2024-04,97.929.213/0001-34,2024-04-01,11.737556,84371140.12,2,NaN
2356962,2024-05,97.929.213/0001-34,2024-05-02,11.779429,84672127.66,2,NaN
2401082,2024-06,97.929.213/0001-34,2024-06-03,11.842104,84763705.25,2,NaN
2462012,2024-07,97.929.213/0001-34,2024-07-01,11.897204,85158100.68,2,NaN


In [6]:
# Drop null
cvm_return = cvm_return.dropna()

## Register data

In [7]:
# Load dataframes
cvm_register = pd.read_csv('cvm_register.csv')
legacy_register = pd.read_csv('legacy_register.csv')

C:\Users\anaak\AppData\Local\Temp\ipykernel_22028\3284308580.py:2: DtypeWarning: Columns (15,18,19,21,23,25,28,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  cvm_register = pd.read_csv('cvm_register.csv')
C:\Users\anaak\AppData\Local\Temp\ipykernel_22028\3284308580.py:3: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,57,58,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  legacy_register = pd.read_csv('legacy_register.csv')


In [8]:
# Check if headers from current and legacy register dataframes are the same
print('CVM register shape:',cvm_register.shape)
print('Legacy register shape:',legacy_register.shape)
print('\n')

# Find which columns are convergent and divergent between them
reg_cur_col = cvm_register.columns
reg_lag_col = legacy_register.columns

# Create an object for columns in common and columns in legacy and not current
common_cols = reg_cur_col.intersection(reg_lag_col)
cur_not_lag = reg_cur_col.difference(reg_lag_col)
lag_not_cur = reg_lag_col.difference(reg_cur_col)

print(common_cols.nunique(),'columns in COMMON between current and legacy:')
print(common_cols)
print('\n')
print(cur_not_lag.nunique(),'columns in CURRENT register dataframe and not in LEGACY:')
print(cur_not_lag)
print('\n')
print(lag_not_cur.nunique(),'columns in LEGACY register dataframe and not in CURRENT:')
print(lag_not_cur)

CVM register shape: (79858, 42)
Legacy register shape: (1841202, 65)


30 columns in COMMON between current and legacy:
Index(['Unnamed: 0', 'CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT',
       'DT_INI_SIT', 'DT_INI_EXERC', 'DT_FIM_EXERC', 'CLASSE', 'DT_INI_CLASSE',
       'RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO',
       'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_ADM', 'INF_TAXA_ADM', 'DIRETOR',
       'CNPJ_ADMIN', 'ADMIN', 'PF_PJ_GESTOR', 'CPF_CNPJ_GESTOR', 'GESTOR',
       'CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',
       'CNPJ_CONTROLADOR', 'CONTROLADOR'],
      dtype='object')


12 columns in CURRENT register dataframe and not in LEGACY:
Index(['CD_CVM', 'CLASSE_ANBIMA', 'DT_CANCEL', 'DT_CONST', 'DT_INI_ATIV',
       'DT_PATRIM_LIQ', 'ENTID_INVEST', 'INF_TAXA_PERFM', 'INVEST_CEMPR_EXTER',
       'TAXA_PERFM', 'TP_FUNDO', 'VL_PATRIM_LIQ'],
      dtype='object')


35 columns in LEGACY register dataframe and not in CURRENT:
Index(['DENOM_COMERC', 'DS_T

### Comparing dataframes
On this step I noticed a difference between columns within current and legacy register dataframe, and why is that?
- According to CVM notes, they changed the infos required from funds along the years. This can happen due a law change or CVM don't see the need to ask for that information (e.g: legacy  'DT_INI_TAXA_ADM' has the date for when administration fee was charged)
- Some headers changed their name. Due to operational reasons, CVM data managers change the name of the columns to fit their system (e.g: legacy 'VL_TAXA_PERFM' and current 'TAXA_PERFM'. Both them have the performance rate for managers based on funds gains)
- New columns addition. With the law change, CVM may ask for new information (e.g: current 'CLASSE_ANBIMA')

Based on those difference, I'll work with the columns they have in common. In total, there are **29 in common columns** I can work with, but not all data in needed.

**Columns in common between current and legacy register:**

*'CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT', 'DT_INI_SIT',
'DT_INI_EXERC', 'DT_FIM_EXERC', 'CLASSE', 'DT_INI_CLASSE',
'RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO',
'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_ADM', 'INF_TAXA_ADM', 'DIRETOR',
'CNPJ_ADMIN', 'ADMIN', 'PF_PJ_GESTOR', 'CPF_CNPJ_GESTOR', 'GESTOR',
'CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',
'CNPJ_CONTROLADOR', 'CONTROLADOR'*

**The data I'll need to identify investment funds are:**

*'CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT','CLASSE', 'DT_INI_CLASSE', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO','CPF_CNPJ_GESTOR', 'GESTOR','CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',*

Based on CVMs [dictionary](https://dados.cvm.gov.br/dados/FI/CAD/META/meta_cad_fi.txt), I'll use the following cols:
| Column | Description|
| --- | ---|
|CNPJ_FUNDO| Investment fund register code |
|DENOM_SOCIAL| Investment fund name |
|DT_REG| Register date|
|SIT| Situation (Active, Deactive)|
|CONDOM| Open/Close fund|
|FUNDO_COTAS| If fund has quotas or not|
|FUNDO_EXCLUSIVO| Exclusive fund|
|CPF_CNPJ_GESTOR| Manager register code|
|GESTOR| Manager name|
|CNPJ_AUDITOR| Audit firm register code|
|AUDITOR| Audit firm name|
|CNPJ_CUSTODIANTE| Issuer register code|
|CUSTODIANTE| Issuer name|

These columns will give me an idea of each fund structue. Those columns will tell me what type of assets each fund is working with, when they started trading those, who is the manager choosing the assets and who is issuing the quotas. And why does it matter?

I can analyze funds performance over the years and identify if a manager has better results than the others. Funds features (such as open/closed or exclusive/not) may indicate better performing funds due to private information or access to better assets. Issuer can hold a specific type of asset or only issue for a certain type of investor, and therefore have a different performance. By keeping audit firm data, I can point who are the big firms working with investment funds or a specific type of asset.

In [9]:
# Merging current and legacy register dataframes based on the columns I selected
main_cols = ['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO','CPF_CNPJ_GESTOR', 'GESTOR','CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE']
cvm_complete_reg = cvm_register[main_cols].copy()
pd.concat([cvm_register,legacy_register], join='inner')
cvm_complete_reg.reset_index(inplace=True)
cvm_complete_reg.drop('index',axis=1, inplace=True)

del cvm_register,legacy_register

cvm_complete_reg.shape

(79858, 13)

In [10]:
# Check data type
print(cvm_complete_reg.info())

# Convert 'DT_REG' and 'DT_INI_CLASSE' in datetime variables
cvm_complete_reg['DT_REG'] = pd.to_datetime(cvm_complete_reg['DT_REG'], format='ISO8601')   # ISO8601 sets date format to Year/month/day

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79858 entries, 0 to 79857
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CNPJ_FUNDO        79858 non-null  object
 1   DENOM_SOCIAL      79858 non-null  object
 2   DT_REG            79858 non-null  object
 3   SIT               79858 non-null  object
 4   CONDOM            66203 non-null  object
 5   FUNDO_COTAS       66557 non-null  object
 6   FUNDO_EXCLUSIVO   55989 non-null  object
 7   CPF_CNPJ_GESTOR   52719 non-null  object
 8   GESTOR            52719 non-null  object
 9   CNPJ_AUDITOR      51990 non-null  object
 10  AUDITOR           51990 non-null  object
 11  CNPJ_CUSTODIANTE  51164 non-null  object
 12  CUSTODIANTE       51164 non-null  object
dtypes: object(13)
memory usage: 7.9+ MB
None


In [11]:
# Check final result
print(cvm_complete_reg.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79858 entries, 0 to 79857
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CNPJ_FUNDO        79858 non-null  object        
 1   DENOM_SOCIAL      79858 non-null  object        
 2   DT_REG            79858 non-null  datetime64[ns]
 3   SIT               79858 non-null  object        
 4   CONDOM            66203 non-null  object        
 5   FUNDO_COTAS       66557 non-null  object        
 6   FUNDO_EXCLUSIVO   55989 non-null  object        
 7   CPF_CNPJ_GESTOR   52719 non-null  object        
 8   GESTOR            52719 non-null  object        
 9   CNPJ_AUDITOR      51990 non-null  object        
 10  AUDITOR           51990 non-null  object        
 11  CNPJ_CUSTODIANTE  51164 non-null  object        
 12  CUSTODIANTE       51164 non-null  object        
dtypes: datetime64[ns](1), object(12)
memory usage: 7.9+ MB
None


In [12]:
# Check to see if I got the current data right
cvm_complete_reg.isna().sum()/len(cvm_complete_reg)

CNPJ_FUNDO          0.000000
DENOM_SOCIAL        0.000000
DT_REG              0.000000
SIT                 0.000000
CONDOM              0.170991
FUNDO_COTAS         0.166558
FUNDO_EXCLUSIVO     0.298893
CPF_CNPJ_GESTOR     0.339841
GESTOR              0.339841
CNPJ_AUDITOR        0.348969
AUDITOR             0.348969
CNPJ_CUSTODIANTE    0.359313
CUSTODIANTE         0.359313
dtype: float64

In [13]:
# Analyze which datapoints are empty, I'll use as reference the issuer feature (CUSTODIANTE) since it has the biggest percentage of nulls.
empty_reg = cvm_complete_reg[cvm_complete_reg['CUSTODIANTE'].isna() == True].sort_values(by='DT_REG', ascending=True)

# See dates with the most null values
empty_reg['DT_REG'].value_counts()/len(empty_reg)

DT_REG
2003-04-30    0.311250
2005-03-21    0.004775
2005-03-16    0.004670
2005-03-18    0.004600
2005-04-15    0.003973
                ...   
2024-01-23    0.000035
1969-12-23    0.000035
1969-12-15    0.000035
1969-10-30    0.000035
1969-10-14    0.000035
Name: count, Length: 5280, dtype: float64

In [14]:
print(empty_reg[empty_reg['DT_REG'] >= pd.to_datetime('2003-04-30')].count())

CNPJ_FUNDO          26168
DENOM_SOCIAL        26168
DT_REG              26168
SIT                 26168
CONDOM              15233
FUNDO_COTAS         15422
FUNDO_EXCLUSIVO     13694
CPF_CNPJ_GESTOR      1573
GESTOR               1573
CNPJ_AUDITOR         1046
AUDITOR              1046
CNPJ_CUSTODIANTE        0
CUSTODIANTE             0
dtype: int64


In [15]:
# Check how many of empty values are fropm canceled funds
empty_reg[empty_reg['SIT'] == 'CANCELADA'].count()/len(empty_reg)

CNPJ_FUNDO          0.983272
DENOM_SOCIAL        0.983272
DT_REG              0.983272
SIT                 0.983272
CONDOM              0.516415
FUNDO_COTAS         0.525859
FUNDO_EXCLUSIVO     0.475117
CPF_CNPJ_GESTOR     0.043284
GESTOR              0.043284
CNPJ_AUDITOR        0.022095
AUDITOR             0.022095
CNPJ_CUSTODIANTE    0.000000
CUSTODIANTE         0.000000
dtype: float64

My null values analysis show that I have aound 36% of missing data for my issuer features. Diving deep into why I detected it can happen if the fund ceases its activities ('CANCELADA') or is in the process of opening. This means that neither of those funds will provide me insight on my return analysis.

To clean my register dataset, I'll remove rows with empty issuer.

But won't that be a problem? You'll lose 36% of your data!
- No. Since I'm focused on funds return, if a fund is not operational (CENCELADA) or in process to begin its operations, I'm not expecting to have historical data on their performance (26k out of 28k null values). I acept to work with a missing value data of 2k.
 

In [16]:
cvm_complete_reg = cvm_complete_reg.dropna()
# Check for duplicates in register file
cvm_complete_reg.duplicated(['CNPJ_FUNDO']).value_counts()

### This means I have 1689 duplicated values in my register dataframe, why?

False    40562
True      1696
Name: count, dtype: int64

In [17]:
duplicated_register = cvm_complete_reg[cvm_complete_reg.duplicated(subset='CNPJ_FUNDO', keep=False)]
duplicated_register.sort_values(by=['CNPJ_FUNDO'], inplace=True)
duplicated_register.head(10)

C:\Users\anaak\AppData\Local\Temp\ipykernel_22028\2568448804.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated_register.sort_values(by=['CNPJ_FUNDO'], inplace=True)


,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE
5731,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5732,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5733,01.392.021/0001-62,BRADESCO H FC DE FUNDOS DE INVESTIMENTO RENDA ...,2005-04-07,EM FUNCIONAMENTO NORMAL,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5734,01.392.021/0001-62,BRADESCO H FC DE FUNDOS DE INVESTIMENTO RENDA ...,2005-04-07,EM FUNCIONAMENTO NORMAL,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5845,01.603.578/0001-03,ICATU VANGUARDA XI FUNDO DE INVESTIMENTO RENDA...,2005-03-23,EM FUNCIONAMENTO NORMAL,Aberto,N,N,42.283.770/0001-39,ICATU SEGUROS S/A,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,60.746.948/0001-12,BANCO BRADESCO S.A.
5846,01.603.578/0001-03,ICATU VANGUARDA XI FUNDO DE INVESTIMENTO RENDA...,2005-03-23,EM FUNCIONAMENTO NORMAL,Aberto,N,N,68.622.174/0001-20,ICATU VANGUARDA GESTÃO DE RECURSOS LTDA,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,60.746.948/0001-12,BANCO BRADESCO S.A.
5852,01.606.509/0001-45,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5853,01.606.509/0001-45,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
6038,01.873.685/0001-43,PREVI RENDA FIXA TÍTULOS PÚBLICOS HTM FUNDO DE...,2005-04-11,EM FUNCIONAMENTO NORMAL,Aberto,N,S,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.
6039,01.873.685/0001-43,PREVI RENDA FIXA TÍTULOS PÚBLICOS HTM FUNDO DE...,2005-04-11,EM FUNCIONAMENTO NORMAL,Aberto,N,S,33.754.482/0001-24,CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO...,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.


My duplicate analysis show an interesting behavior. Usually, analysts drop/delete duplicates in their datasets. The reason is because duplicates usually are viewed as an error. However, when we see what is happening in our register dataframe, I see that funds are registered two times (at least): first when the fund is opened to business and the second when the fund closes its activities.

Based on this behavior, I can't drop duplicates because in this case it is a feature rather than an error. Let's see if there are duplicated values even if the fund still in business.

In [18]:
fund_active = duplicated_register[~duplicated_register['SIT'].str.contains('CANCELADA')]
fund_active = fund_active[fund_active.duplicated(subset='CNPJ_FUNDO', keep=False)]
fund_active.head(10)

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE
5731,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5732,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5733,01.392.021/0001-62,BRADESCO H FC DE FUNDOS DE INVESTIMENTO RENDA ...,2005-04-07,EM FUNCIONAMENTO NORMAL,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5734,01.392.021/0001-62,BRADESCO H FC DE FUNDOS DE INVESTIMENTO RENDA ...,2005-04-07,EM FUNCIONAMENTO NORMAL,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5845,01.603.578/0001-03,ICATU VANGUARDA XI FUNDO DE INVESTIMENTO RENDA...,2005-03-23,EM FUNCIONAMENTO NORMAL,Aberto,N,N,42.283.770/0001-39,ICATU SEGUROS S/A,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,60.746.948/0001-12,BANCO BRADESCO S.A.
5846,01.603.578/0001-03,ICATU VANGUARDA XI FUNDO DE INVESTIMENTO RENDA...,2005-03-23,EM FUNCIONAMENTO NORMAL,Aberto,N,N,68.622.174/0001-20,ICATU VANGUARDA GESTÃO DE RECURSOS LTDA,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,60.746.948/0001-12,BANCO BRADESCO S.A.
5852,01.606.509/0001-45,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Aberto,S,S,51.990.695/0001-37,BRADESCO VIDA E PREVIDÊNCIA S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
5853,01.606.509/0001-45,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI ...,2005-04-09,EM FUNCIONAMENTO NORMAL,Aberto,S,S,60.746.948/0001-12,BANCO BRADESCO S.A.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,60.746.948/0001-12,BANCO BRADESCO S.A.
6038,01.873.685/0001-43,PREVI RENDA FIXA TÍTULOS PÚBLICOS HTM FUNDO DE...,2005-04-11,EM FUNCIONAMENTO NORMAL,Aberto,N,S,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.
6039,01.873.685/0001-43,PREVI RENDA FIXA TÍTULOS PÚBLICOS HTM FUNDO DE...,2005-04-11,EM FUNCIONAMENTO NORMAL,Aberto,N,S,33.754.482/0001-24,CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO...,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.


This results show that a fund can be registered twice even if it still active. However, the condition of how the funding is running might be different. A fund can still be operational, but in a specific state, such as in procress of registering, on hold for analysis, or even starting the process of liquidation (selling assets to end the fund).

Based on these condition, I need to know what can I expect to see withing the status ('SIT') feature.

In [19]:
# Find unique labels for each situation that are not canceled (CANCELADA)
print(fund_active['SIT'].unique())
# Delete variables to clean memory space
del fund_active,duplicated_register

['EM FUNCIONAMENTO NORMAL' 'FASE PRÉ-OPERACIONAL' 'LIQUIDAÇÃO']


Status feature ('SIT') has 3 unique labels:

- 'EM FUNCIONAMENTO NORMAL': operating business as usual
- 'FASE PRÉ-OPERACIONAL': 1 phase before opening to business
- 'LIQUIDAÇÃO': in process of selling assets to end the fund

These classes are important to tell us the story of each fund. So a fund can be registered twice even if it's not stopped its activities.

But status ('SIT') is not the only feature that can explain duplicated registers. Look for example what happend with fund 32.891.432/0001-26. All its features are the same, except for a change Manager ('GESTOR'), which means the fund at some point in time changed institutions managing their assets and had to issue a new registration.

In [20]:
# Now I'll take care of all categorical variables in my dateset
cvm_complete_reg = pd.get_dummies(cvm_complete_reg, columns = ['SIT', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO'], dtype = 'int')

Now, I'll create a new columns/variable for the issuer. Retails banks can create funds, manage the assets and issue quotas, while using their advantage of having a huge client. Those clients come from their operations in credit market, while other players are smaller or have their business purely investment. I'll use [Valor 100](https://infograficos.valor.globo.com/valor1000/rankings/os-100-maiores-bancos/2023) report, to select the top 10 retail banks in Brazil due their capitalization and total assets in Reais (BRL). 

In [21]:
# Create list of retail banks in Brazil
retail_bank = ['SANTANDER', 'ITAU', 'BANCO DO BRASIL', 'BB', 'BRADESCO', 'CAIXA', 'SAFRA', 'SICREDI','SICOOB', 'CITIBANK']

# Create a new dummy variable if the issuer is a retail bank
def retail_find(retail_name):
    for word in retail_bank:
        if word in retail_name:
            return 1
    return 0

cvm_complete_reg['retail_bank_manager'] = cvm_complete_reg['GESTOR'].apply(retail_find)
cvm_complete_reg['retail_bank_issuer'] = cvm_complete_reg['CUSTODIANTE'].apply(retail_find)

# Label the reatil bank, I'll use this variable only in EDA

## Create a new dummy variable if the issuer is a retail bank
def retail_name(retail_name):
    for word in retail_bank:
        if word in retail_name:
            return word
    return None

cvm_complete_reg['manager_name'] = cvm_complete_reg['GESTOR'].apply(retail_name)
cvm_complete_reg['issuer_name'] = cvm_complete_reg['CUSTODIANTE'].apply(retail_name)

# Drop columns I won't use related to issuer
cvm_complete_reg = cvm_complete_reg.drop(columns=['CPF_CNPJ_GESTOR', 'GESTOR','CNPJ_CUSTODIANTE', 'CUSTODIANTE'], axis=1)

Now, I need to enconde auditors variable. In auditors business, there are some big companies that used to monopolize the market. They were called [Big 4](https://auditoria.fecap.br/blog/big-four-conheca-as-principais-empresas-de-auditoria/) (Delloitte, PwC, EY and KPMG). I'll enconde my auditor variable for if it belongs to the Big 4.

In [22]:
# Do the same for issuer and auditor 
## Auditor
big4 = ['DELLOITE', 'ERNST & YOUNG', 'PRICEWATERHOUSECOOPERS', 'KPMG']

# Create function to encode string
def big4_find(auditor):
    for word in big4:
        if word in auditor:
            return 1
    return 0

# Create function to return which Big4
def big4_name(auditor):
    for word in big4:
        if word in auditor:
            return word
    return None

# Apply on register dataframe
cvm_complete_reg['big4'] = cvm_complete_reg['AUDITOR'].apply(big4_find)
cvm_complete_reg['big4_name'] = cvm_complete_reg['AUDITOR'].apply(big4_name)

# Drop used columns
cvm_complete_reg = cvm_complete_reg.drop(columns=['CNPJ_AUDITOR', 'AUDITOR'], axis=1)

# Check final register dataset
# cvm_complete_reg.info()

## Macro data

In [60]:
# Load datasets
current_ipca = pd.read_csv('current_ipca.csv')
current_rf = pd.read_csv('current_rf.csv')

In [61]:
# Check inflation dataset
current_ipca['Date'] = pd.to_datetime(current_ipca['Date'])
current_ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    56 non-null     datetime64[ns]
 1   IPCA    56 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.0 KB


In [62]:
# Check risk-free dataset
current_rf['Date'] = pd.to_datetime(current_rf['Date'])
current_rf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1725 non-null   datetime64[ns]
 1   selic   1725 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 27.1 KB


In [63]:
# Create key to match with other datasets
current_ipca['year_month'] = current_ipca['Date'].dt.to_period('M')
current_rf['year_month'] = current_rf['Date'].dt.to_period('M')

# Drop date
current_ipca = current_ipca.drop('Date', axis=1)
current_rf = current_rf.drop('Date', axis=1)

Now, I need to adjust my rates to the same. Using Brazilian Central Bank (BACEN) API, I discovered that my inflation rate is using a monthly base while my risk-free is in an annual base. I'll transform my monthly inflation rate into an annual inflation rate to match my risk-free.

But Ana, do you really need to change rates base? If you just standardize it during dataset preparation won't fix it? **NO**
- In finance we need to work with all our rates in the same base to grand comparability. Standardizing fix a scale issue, not a fundamental issue with our data.


In [69]:
# Converting indlation monthly rates into annual rates
## Before converting, just need to adjust the rate from % to decimal
current_ipca['IPCA_aa'] = (((1 + (current_ipca['IPCA']/100))**12)-1)*100
current_ipca = current_ipca.drop(columns=['IPCA'], axis=1)

## Market data

In [72]:
# Load datasets
dolar_mkt = pd.read_csv('dolar_mkt.csv')
ibov_mkt = pd.read_csv('ibov_mkt.csv')

In [29]:
# Type of data
print(dolar_mkt.info())
print('\n')
print(ibov_mkt.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195 entries, 0 to 1194
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1195 non-null   object 
 1   Open       1195 non-null   float64
 2   High       1195 non-null   float64
 3   Low        1195 non-null   float64
 4   Close      1195 non-null   float64
 5   Adj Close  1195 non-null   float64
 6   Volume     1195 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 65.5+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139 entries, 0 to 1138
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1139 non-null   object 
 1   Open       1139 non-null   float64
 2   High       1139 non-null   float64
 3   Low        1139 non-null   float64
 4   Close      1139 non-null   float64
 5   Adj Close  1139 non-null   float64
 6   Volume     1139 non-null   int64 

Looking our market datasets, seems like all variables have the type they should have except for date. After fixing the date, I'll keep only the Adjusted Close in each dataset.

I'll also add the key column to match market data with my main dataframe, year_month.

**Why are you using Adjusted Close?**
- Adjusted close is the price of the asset by the end of the day, accounted for all variations due splits or dividend distributions. For currency (FX) we don't have dividends or splits, so in theory you could use the regular close value. However, as good practice of financial research, I'm sticking with adjusted close.

In [73]:
# Changing column  'Date' to datetime column
dolar_mkt['Date'] = pd.to_datetime(dolar_mkt['Date'])
ibov_mkt['Date'] = pd.to_datetime(ibov_mkt['Date'])

# Keeping only adjusted close value and date
dolar_mkt = dolar_mkt[['Date', 'Adj Close']]
ibov_mkt = ibov_mkt[['Date', 'Adj Close']]

# Add key column to match other datasets
dolar_mkt['year_month'] = dolar_mkt['Date'].dt.to_period('M')
ibov_mkt['year_month'] = ibov_mkt['Date'].dt.to_period('M')

# Drop date column
dolar_mkt = dolar_mkt.drop('Date', axis=1)
ibov_mkt = ibov_mkt.drop('Date', axis=1)

# Rename columns
dolar_mkt = dolar_mkt.rename(columns={"Adj Close": "Dolar_Adj_Close"})
ibov_mkt = ibov_mkt.rename(columns={"Adj Close": "Ibov_Adj_Close"})

## Merging datasets:
Now it's time to merge our dataframes, return base,. macroeconomic, market and register data. Due to my duplicate analysis, I need to pay attention to how to merge data. For instance, a fund can have two different managers institution based on the date.

So my keys to merge the data will be the funds registration number ('CNPJ_FUNDO') and the register data ('DT_REG'). Naturally, another issue will appear: return data is in a daily frequency and register data is ponctual. This means I'll have a series of empty rows for the features coming from register dataframe. I'll fix it by coping each fund previous row, this means: since register occurs only when the fund is created or has some change, it's fair to use the data on the register/change day to further dates until the fund dissapears from our database (meaning they are ceased their operations).

In [74]:
# Merge macroeconomic and market dataframe
current_ipca = current_ipca[['year_month', 'IPCA_aa']]
support_df = pd.merge(current_rf, current_ipca, on='year_month', how='left')
support_df = pd.merge(support_df, dolar_mkt, on='year_month', how='left')
support_df = pd.merge(support_df, ibov_mkt, on='year_month', how='left')

# Save support dataset
support_df.to_csv('support_df.csv')

In [33]:
# Merg key
## Substitute all dates before 2020 to '2020-01-01', this way I can merge datasets
df_reg = cvm_complete_reg
df_reg.sort_values(by='DT_REG', inplace=True, ascending=False)
df_reg.loc[df_reg['DT_REG'] < '2020-01-01','DT_REG'] = pd.to_datetime('2020-01-01')
df_reg = df_reg.drop_duplicates(subset=['CNPJ_FUNDO','DT_REG'], keep='first')  # Drop older data
df_reg['year_month'] = df_reg['DT_REG'].dt.to_period('M')   # I'm gonna use year-month as a key

C:\Users\anaak\AppData\Local\Temp\ipykernel_22028\2044193569.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg['year_month'] = df_reg['DT_REG'].dt.to_period('M')   # I'm gonna use year-month as a key


In [34]:
# Merge return and register dataframes based on register year-month and fund ID
fund_df = pd.merge(cvm_return, df_reg, how='left', on=['year_month','CNPJ_FUNDO'])

# Possible issue: fund register in a month and only starts to operate in the following month (on which my previous merge won't work)
fund_df.sort_values(by=['CNPJ_FUNDO','year_month'], inplace=True)

fund_df = fund_df.fillna(method='ffill')
# fund_df

C:\Users\anaak\AppData\Local\Temp\ipykernel_22028\3632909830.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fund_df = fund_df.fillna(method='ffill')


In [35]:
# Check if ffill method worked
fund_list = fund_df[['CNPJ_FUNDO', 'DENOM_SOCIAL']].drop_duplicates()
reg_list = df_reg[['CNPJ_FUNDO', 'DENOM_SOCIAL']].drop_duplicates()

In [36]:
# Compare final dataset and original register dataframe
reg_list = reg_list.rename(columns={'DENOM_SOCIAL':'DENOM_SOCIAL_REG'})
check = pd.merge(fund_list,reg_list, on='CNPJ_FUNDO', how='left')
check['correct_name'] = check['DENOM_SOCIAL'] == check['DENOM_SOCIAL_REG']
check['correct_name'].value_counts()

correct_name
True     21548
False    12560
Name: count, dtype: int64

In [37]:
check[check['correct_name'] == False]

,CNPJ_FUNDO,DENOM_SOCIAL,DENOM_SOCIAL_REG,correct_name
28,00.754.543/0001-02,BANRISUL VIP FUNDO DE INVESTIMENTO RENDA FIXA ...,NaN,False
56,00.826.600/0001-03,ITAÚ BASIS OPEN RENDA FIXA REFERENCIADO DI FUN...,NaN,False
62,00.828.371/0001-66,FATOR MAX CORPORATIVO FUNDO DE INVESTIMENTO EM...,NaN,False
84,00.836.255/0001-99,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,NaN,False
106,00.902.484/0001-64,FAPA SENIOR FUNDO DE INVESTIMENTO MULTIMERCADO,NaN,False
...,...,...,...,...
34012,56.154.549/0001-95,VINCI CRÉDITO INFRA II MASTER FUNDO INCENTIVAD...,G5 TOKYO FIF RF INCENTIVADO DE INVESTIMENTO EM...,False
34013,56.159.439/0001-16,VINCI CRÉDITO INFRA II MASTER FUNDO INCENTIVAD...,ZETA FUNDO INCENTIVADO DE INVESTIMENTO EM INFR...,False
34014,56.171.380/0001-81,VINCI CRÉDITO INFRA II MASTER FUNDO INCENTIVAD...,TALAL FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDI...,False
34015,56.176.497/0001-58,VINCI CRÉDITO INFRA II MASTER FUNDO INCENTIVAD...,G5 MACAMA II FUNDO DE INVESTIMENTO MULTIMERCADO,False


In [38]:
# Check on original regsiter dataframe if the fund exist
cvm_complete_reg[cvm_complete_reg['CNPJ_FUNDO'] == '00.754.543/0001-02']

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT_CANCELADA,SIT_EM FUNCIONAMENTO NORMAL,SIT_FASE PRÉ-OPERACIONAL,SIT_LIQUIDAÇÃO,CONDOM_Aberto,CONDOM_Fechado,FUNDO_COTAS_N,FUNDO_COTAS_S,FUNDO_EXCLUSIVO_N,FUNDO_EXCLUSIVO_S,retail_bank_manager,retail_bank_issuer,manager_name,issuer_name,big4,big4_name


In [39]:
# Check on original fund dataframe if there is any transation
fund_df[fund_df['CNPJ_FUNDO'] == '00.754.543/0001-02'].tail(3)

,year_month,CNPJ_FUNDO,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST,quota_return,DENOM_SOCIAL,DT_REG,SIT_CANCELADA,...,FUNDO_COTAS_N,FUNDO_COTAS_S,FUNDO_EXCLUSIVO_N,FUNDO_EXCLUSIVO_S,retail_bank_manager,retail_bank_issuer,manager_name,issuer_name,big4,big4_name
1554,2020-12,00.754.543/0001-02,2020-12-31,5.326529,6638328.10,1,0.009802,BANRISUL VIP FUNDO DE INVESTIMENTO RENDA FIXA ...,2020-01-01,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,ITAU,BRADESCO,0.0,ERNST & YOUNG
1555,2021-01,00.754.543/0001-02,2021-01-29,5.333709,6647276.05,1,0.006896,BANRISUL VIP FUNDO DE INVESTIMENTO RENDA FIXA ...,2020-01-01,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,ITAU,BRADESCO,0.0,ERNST & YOUNG
1556,2021-02,00.754.543/0001-02,2021-02-09,5.334434,6648179.13,1,5.334434,BANRISUL VIP FUNDO DE INVESTIMENTO RENDA FIXA ...,2020-01-01,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,ITAU,BRADESCO,0.0,ERNST & YOUNG


Seems like not all my matches are perfect between register and financial records. This happens because our legacy data is incomplete. These funds were registered long time ago, when CVM didn't ask for all agents data (like issuers). Since issuer seems to be an important variable to discusds in my exploratory data analysis (EDA), I'll keep it for now and drop when I'm testing different models.

So for my EDA, all missmatches will be treated as 'Other'. 

In [40]:
# Merge column to know which registers are wrong
check = check[['CNPJ_FUNDO', 'correct_name']]
check['correct_name'] =  check['correct_name'].astype(bool)
fund_df = pd.merge(fund_df, check, on='CNPJ_FUNDO', how='left')

In [41]:
# Change the value to 0 if the register if wrong (correct_name =  False)
fund_df['retail_bank_manager'] = fund_df['retail_bank_manager'].where(fund_df['correct_name'], 0)
fund_df['retail_bank_issuer'] = fund_df['retail_bank_issuer'].where(fund_df['correct_name'], 0)
fund_df['big4'] = fund_df['big4'].where(fund_df['correct_name'], 0)

# Change the value to 'Other' if the register if wrong (correct_name =  False)
fund_df['manager_name'] = fund_df['manager_name'].where(fund_df['correct_name'], 'Other')
fund_df['issuer_name'] = fund_df['issuer_name'].where(fund_df['correct_name'], 'Other')
fund_df['big4_name'] = fund_df['big4_name'].where(fund_df['correct_name'], 'Other')

# Delete complete register df to open up space (I also don't need it anymore)
del cvm_complete_reg

## Feature engineering:
Now, I have all my basic variables in my dataframe. But this is not enough, I need to adjust a few things before jumping to our models.

1. Lag numerical variables 1 period: This is a classic covariance issue in finance. I'll give an example to make it easier:
    - On January, a fund had X% return to investors. In Januray, we observe an increase in the value of total assets, number of investors buying quotas and quota value. Now, if fund return is based on quota value at the end of the month minus the value at the beginning of the month, our return increase was caused by the new comers or because the assets valued? 
    - By lagging one period, all assets and changes caused by new investors won't causa artificial changes in quota current value, since all changes already happened last month.

2. After lagging my variables, I will create a risk measure called Sharpe Ration. This index is used in finance to analyze how is your return compared to the risk you are taking. I expect to see funds with higher returns with a higher Sharpe ratio.

In [42]:
# Lagging variables by fund and date
# Create list of variables that will be lagged
lag_var = ['VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']

# Sort dataframe to lag
fund_df.sort_values(by=['CNPJ_FUNDO', 'year_month'], inplace=True)

# Lag variables
for variable in lag_var:
    fund_df[f'{variable}_lag1'] = fund_df.groupby('CNPJ_FUNDO')[variable].shift(1)


# Drops empty row due to lag in variables
fund_df = fund_df.dropna()


In [43]:
# Check for outliers
# fund_df[(np.abs(stats.zscore(fund_df)) < 3).all(axis=1)]

In [44]:
# Save csv file with data
fund_df.to_csv('fund_df.csv')